In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#default_exp classify_image

In [41]:
#export
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np
from food.psql import *

# !nbdev_build_lib
import requests
from food.clipmodel import image2clip,text2clip
from PIL import Image

In [4]:
#export
from food.milvus import collection
collection.load()
collection.num_entities

7083

In [5]:
#export
def search_by_clip(clip, topk=5, query_expr='',search_params = {}): #32768
    search_params = {"metric_type": "IP",
                     "params": search_params}

    print(query_expr)
    results = collection.search(clip[None,], "clip", search_params, limit=topk, expr=query_expr, output_fields=["id"])
    results = results[0]
    return list(results.distances), results.ids

def get_metadata(ids):
    q = f"""select * from foods where id in {tuple(ids)}"""
    return pd.read_sql(q,engine)

In [194]:
#export
def search_image(url=None,path=None):
    if url:
        response = requests.get(url, stream=True)
        image = Image.open(response.raw)
    elif path:
        image = Image.open(path)
    clip = image2clip(image)
    results = search_by_clip(clip.numpy())
    df = get_metadata(results[1])
    df['score'] = results[0]
    return df

In [189]:
#export
def get_calories(url=None,path=None):
    r = search_image(url=url)
    r = r[(r['energy_kcal_100g']>r['energy_kcal_100g'].quantile(0.3))&(r['energy_kcal_100g']<r['energy_kcal_100g'].quantile(0.9))]
    rr = r[['energy_kcal_100g','proteins_100g','fat_100g','carbohydrates_100g']].mean().round(2)
    rr['ids'] =  r['id'].tolist()
    rr['scores'] =  r['score'].tolist()
    rr['names']  =  r['product_name'].tolist()
    return rr

In [193]:
url = 'https://s3-ap-south-1.amazonaws.com/fooddata.image/12293.jpg'

response = requests.get(url, stream=True)
image = Image.open(response.raw)
image_clip = image2clip(image)

In [205]:
#milvus search
search_image(url)

,id,description,category,energy,protein,carb,fat,clip,score
0,1101144,"Cobbler, apple",Cakes and pies,194.0,2.31,36.38,4.95,"[0.03982123, -0.021529237, 0.030978544, -0.001...",0.255718
1,1101153,"Fritter, apple","Doughnuts, sweet rolls, pastries",381.0,6.46,34.75,24.46,"[0.057608202, 0.031312082, 0.02672885, -0.0110...",0.247006
2,1102644,"Apple, raw",Apples,52.0,0.26,13.81,0.17,"[0.03289434, 0.009451146, 0.04302109, -0.00526...",0.246747
3,1102782,"Apples, baby food, toddler",Baby food: fruit,50.0,0.19,11.56,0.36,"[0.0003478544, -0.022820584, 0.033641424, 0.00...",0.245858
4,1102813,"Apples and pears, baby food, junior",Baby food: fruit,40.0,0.15,10.95,0.05,"[0.017789599, 0.0068757446, 0.0164412, 0.01423...",0.243942


In [206]:
cos = torch.nn.CosineSimilarity(dim=-1, eps=1e-6)
foods = pd.read_sql('select * from foods',engine)
clip = torch.Tensor(foods['clip']).cuda(2).type(torch.float16)
foods['score'] = cos(clip, torch.Tensor(image_clip).cuda(2)).cpu().detach().numpy()
truth = foods.sort_values('score',ascending=False).head(50)
truth.head()

,id,description,category,energy,protein,carb,fat,clip,score
5143,1102644,"Apple, raw",Apples,52.0,0.26,13.81,0.17,"[0.03289434, 0.009451146, 0.04302109, -0.00526...",0.255692
5276,1102782,"Apples, baby food, toddler",Baby food: fruit,50.0,0.19,11.56,0.36,"[0.0003478544, -0.022820584, 0.033641424, 0.00...",0.246995
3639,1101144,"Cobbler, apple",Cakes and pies,194.0,2.31,36.38,4.95,"[0.03982123, -0.021529237, 0.030978544, -0.001...",0.246746
3648,1101153,"Fritter, apple","Doughnuts, sweet rolls, pastries",381.0,6.46,34.75,24.46,"[0.057608202, 0.031312082, 0.02672885, -0.0110...",0.245845
5307,1102813,"Apples and pears, baby food, junior",Baby food: fruit,40.0,0.15,10.95,0.05,"[0.017789599, 0.0068757446, 0.0164412, 0.01423...",0.243948


In [ ]:
from food.milvus import collection

In [207]:
collection.load()
collection.num_entities

7083

In [ ]:
res = collection.query(
  expr = "id in [1102644]", 
  output_fields = ["clip"],
  consistency_level="Strong"
)

In [208]:
cos(torch.Tensor(res[0]['clip']),image_clip)

tensor(0.2557)

In [224]:
torch.linalg.norm(image_clip, dim=-1)

tensor(1.0000)

In [228]:
torch.linalg.norm(torch.Tensor(res[0]['clip']), dim=-1)

tensor(1.0000)

In [187]:
search_image(url)

,id,description,category,energy,protein,carb,fat,clip,score
0,1101144,"Cobbler, apple",Cakes and pies,194.0,2.31,36.38,4.95,"[0.03982123, -0.021529237, 0.030978544, -0.001...",0.255718
1,1101153,"Fritter, apple","Doughnuts, sweet rolls, pastries",381.0,6.46,34.75,24.46,"[0.057608202, 0.031312082, 0.02672885, -0.0110...",0.247006
2,1102644,"Apple, raw",Apples,52.0,0.26,13.81,0.17,"[0.03289434, 0.009451146, 0.04302109, -0.00526...",0.246747
3,1102782,"Apples, baby food, toddler",Baby food: fruit,50.0,0.19,11.56,0.36,"[0.0003478544, -0.022820584, 0.033641424, 0.00...",0.245858
4,1102813,"Apples and pears, baby food, junior",Baby food: fruit,40.0,0.15,10.95,0.05,"[0.017789599, 0.0068757446, 0.0164412, 0.01423...",0.243942


In [13]:
get_calories(url)

energy_kcal_100g                                                  588.5
proteins_100g                                                     10.48
fat_100g                                                          49.15
carbohydrates_100g                                                22.06
ids                   [227195, 227314, 546920, 670513, 2910991, 3177...
scores                [0.3648509979248047, 0.36058855056762695, 0.35...
names                 [Lindt Excellence Dark Chocolate Bar - 70%, Li...
dtype: object

In [20]:
pd.read_sql("select count(*) from foods_big where energy_kcal_100g is null",engine)

,count
0,0


In [118]:
pd.read_sql("select * from foods_big where clip is not null",engine)

,id,product_name,keywords,ingredients_text,categories,food_groups,energy_kcal_100g,proteins_100g,fat_100g,carbohydrates_100g,clip


In [48]:
df = pd.read_sql("select * from foods_big",engine).set_index('id')

In [103]:
text = df[['product_name','keywords','ingredients_text','categories','food_groups']].fillna("")
for c in text.columns: text[c] = text[c].str.replace('NaN','')
text = text['product_name']+ '. ' + text['food_groups']+ '. ' + text['keywords'] + '. ' + text['ingredients_text'] + '. '+ text['categories']

In [117]:
text.loc[text.sample(1).index[0]]

"panna cotta fruits rouges. dairy-desserts. panna, de, au, produit, lacte, dessert, rouge, laitier, franprix, framboise, couli, la, fruit, cotta. Crème fraîche 31,7% - eau - sirop de glucose - framboises 10,4% - sucre - lait écrémé en poudre - huile de coco - alcool - amidon de tapioca - épaississants : carraghénanes, alginate de sodium - correcteur d’acidité : acide citrique - gélifiant : pectines - arômes naturels - émulsifiant : mono - et diglycérides d’acides gras - extrait de sureau - conservateur : sorbate de potassium.Teneur en alcool : 1,0%.Traces d'œuf et blé.Les informations en gras sont destinées aux personnes intolérantes ou allergiques.. Produits laitiers, Desserts, Desserts lactés, Panna cottas, Panna cottas au coulis de fruits, Panna cottas à la framboise"

In [14]:
!nbdev_build_lib

Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_tools.ipynb.
Converted 04_milvus.ipynb.
Converted 06_clip.ipynb.
Converted 0_template.ipynb.
Converted bot.ipynb.
Converted classfy_image.ipynb.
Converted inference.ipynb.
Converted milvus_update.ipynb.
Converted openfoodfacts_json_parsing.ipynb.
